# Orbit response 

In [ ]:
import sys
import os
from os.path import join
import collections
from datetime import datetime
import importlib
import h5py
import numpy as np
from pprint import pprint
from tqdm import tqdm
from matplotlib import pyplot as plt
import proplot as pplt

from tools.data import H5Reader
from tools.plotting import linear_fit

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Setup 

In [ ]:
folder = 'Diagnostics/Data/Measurements/orbit_response/2022-05-11/'
filenames = os.listdir(folder)
filenames

In [ ]:
class DataFile(H5Reader):
    def __init__(self, filename, folder='.'):
        super().__init__(filename, folder)
        self.quad, self.screen, self.cam = self.filename.split('-')[-1].split('.h5')[0].split('_')
        self.screen_int = self.screen[-2:]
        self.quad_int = self.quad[-2:]
        self.cam_int = self.cam[-2:]

Create nested dictionary of data files. 

In [ ]:
dfiles = dict()
for filename in filenames:
    df = DataFile(filename, folder)
    if df.screen not in dfiles:        
        dfiles[df.screen] = dict()
    dfiles[df.screen][df.quad] = df

In [ ]:
screens = sorted(dfiles)
print('screens:', screens)
print('dfiles:')
pprint(dfiles)

## Analysis

In [ ]:
screen = 'VS14'
quads = sorted(dfiles[screen].keys(), key=lambda quad: int(quad[-2:]))
print('quads:', quads)

In [ ]:
ncols = 4
nrows = int(np.ceil(len(dfiles[screen]) / ncols))   
remainder = nrows * ncols - len(quads)
istart = 1  # Ignore first data point.
kws = dict(marker='.', ms=3)

fig, axes = pplt.subplots(ncols=ncols, nrows=nrows, figwidth=8.0, sharey=False)
for ax in axes[-remainder:]:
    ax.axis('off')
for ax, quad in zip(axes, quads):
    data = dfiles[screen][quad].data
    name_rb = quad + '_CurrentRb'
    name_set = quad + '_CurrentSet'
    x = np.arange(istart, len(df.data))
    ax.plot(x, data[istart:, name_rb], color='black', label='CurrentRb', **kws)
    ax.plot(x, data[istart:, name_set], color='black', alpha=0.3, label='CurrentSet', **kws)
    ax.set_title(quad)
axes[0, -1].legend(loc='r', ncols=1, framealpha=0)
axes.format(xlabel='Step')

In [ ]:
for screen in screens:
    
    colors = pplt.Cycle('colorblind').by_key()['color']

    fig, axes = pplt.subplots(ncols=ncols, nrows=nrows, figwidth=8.0, sharex=False, sharey=1,
                              hspace=6.0)
    for ax in axes[-remainder:]:
        ax.axis('off')
    for ax, quad in zip(axes, quads):
        data = dfiles[screen][quad].data
        current_rb_pv = quad + '_CurrentRb'
        x = data[current_rb_pv]
        x = x[istart:]
        for i, (dim, yloc, halign) in enumerate(zip(['x', 'y'], [0.98, 0.88], ['left', 'right'])):
            profiles = data['Cam09_Profile' + dim.capitalize()]
            profiles = profiles[istart:]
            means = np.argmax(profiles, axis=1)
            yfit, slope, intercept = linear_fit(x, means)
            ax.plot(x, yfit, color=colors[i], alpha=0.2)
            ax.plot(x[istart:], means[istart:], color=colors[i], marker='.', lw=0, label=dim)
            ax.annotate(f'slope = {slope:.1f}', xy=(0.98, yloc), xycoords='axes fraction',
    #                     fontsize='small',
                        horizontalalignment='right', verticalalignment='top', color=colors[i])
        ax.format(xlabel=current_rb_pv)
    axes[0].legend(loc='top', framealpha=0)
    # Pad limits to fit text.
    frac_pad_t = 0.35
    frac_pad_b = 0.05
    for ax in axes:
        ymin, ymax = ax.get_ylim()
        distance = abs(ymax - ymin)
        pad_t = frac_pad_t * distance
        pad_b = frac_pad_b * distance
        ax.format(ylim=(ymin - pad_b, ymax + pad_t))
    axes.format(ylabel='Position [pixel]', suptitle=f'Screen: {screen}')
    plt.savefig(f'_output/quad_scan_{screen}')
    plt.show()

Everything in one plot.

In [ ]:
istart = 1

In [ ]:
colors = pplt.Cycle('colorblind').by_key()['color']
line_kws = dict(alpha=0.2)
scatter_kws = dict(marker='.', s=40, alpha=0.9, ec='None')

fig, axes = pplt.subplots(ncols=7, nrows=len(screens), figwidth=10.0)
for row, screen in enumerate(screens):
    for col, (ax, quad) in enumerate(zip(axes[row, :], quads)):
        data = dfiles[screen][quad].data
        current_rb_pv = quad + '_CurrentRb'
        x = data[current_rb_pv]
        x = x[istart:]
        x_mean = np.mean(x)
        x_frac_dev = (x - x_mean) / x_mean
#         ax1 = ax.alty(color=colors[1])
#         if col != axes.shape[1] - 1:
#             ax1.format(yticklabels=[])
        for i, (_ax, dim, yloc, halign) in enumerate(zip([ax, ax], ['x', 'y'], [0.98, 0.88], ['left', 'right'])):
            profiles = data['Cam09_Profile' + dim.capitalize()]
            profiles = profiles[istart:]
            means = np.argmax(profiles, axis=1)
            yfit, slope, intercept = linear_fit(x, means)
            _ax.plot(x_frac_dev, yfit, color=colors[i], **line_kws)
            _ax.scatter(x_frac_dev, means, c=colors[i], label=dim, **scatter_kws)
axes[0, 0].legend(loc='upper left')

# Pad limits.
frac_pad_x = 0.05
frac_pad_y = 0.0
for row, ax in enumerate(axes[:, 0]):
    vmin, vmax = ax.get_ylim()
    pad = frac_pad_y * abs(vmax - vmin)
    axes[row, :].format(ylim=(vmin - pad, vmax + pad))
for col, ax in enumerate(axes[-1, :]):
    vmin, vmax = ax.get_xlim()
    if vmin > vmax:
        vmax, vmin = vmin, vmax
    pad = frac_pad_x * abs(vmax - vmin)
    axes[:, col].format(xlim=(vmin - pad, vmax + pad))

axes.format(
    ylabel='Position [pixel]', 
    xlabel='Fractional quad change',
    leftlabels=[f'{screen}' for screen in screens],
    toplabels=quads,
)
plt.savefig(f'_output/quad_scan')
plt.show()